# PYNQ를 이용한 Face Detection 영상 처리 (Zybo-Z7-20 Board)

0. 입력할 영상 길이 및 경로 설정

In [ ]:
# set video's length
frame_cnt = 200

# video file's path
video_file = '/home/xilinx/jupyter_notebooks/video_of_people_walking (Original).mp4'

1. 패키지 불러오기

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

In [ ]:
import cv2
import time

2. Base overlay를 다운받고 HDMI output 설정

In [ ]:
base = BaseOverlay("base.bit")
hdmi_out = base.video.hdmi_out

3. 영상을 불러오고 HDMI 출력 활성화

In [ ]:
# video capturing from video files, image sequences or cameras
videoIn = cv2.VideoCapture(video_file)

w = int(videoIn.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(videoIn.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Frame_Width: "w, "Frame_Height: "h)

# configure hdmi output
mode = VideoMode(w,h,24)
hdmi_out.configure(mode, PIXEL_RGB)
hdmi_out.start()

4. 영상을 불러왔는지 확인

In [ ]:
print("Capture device is open: " + str(videoIn.isOpened()))

5. Load Cascade Classifier Model

In [ ]:
face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('data/haarcascade_eye.xml')

6. 영상 출력 결과 확인

In [ ]:
start_time = time.time()

for _ in range(frame_cnt):

    # Get a frame from the video
    ret, frame = videoIn.read()
    if ret:
      orig_img = frame.copy()
      orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

      gray = cv2.cvtColor(orig_img, cv2.COLOR_BGR2GRAY) # gray image

      faces = face_cascade.detectMultiScale(gray, 1.3, 5) # detect "face"

      for (x,y,w,h) in faces:
          color = (255,0,0)
          cv2.rectangle(orig_img,(x,y),(x+w,y+h),color,2)
          cv2.putText(orig_img, "face", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

          roi_gray = gray[y:y+h, x:x+w] # 탐지한 face 객체 위치설정
          roi_color = orig_img[y:y+h, x:x+w]
          eyes = eye_cascade.detectMultiScale(roi_gray)

          for (ex,ey,ew,eh) in eyes:
              cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

      # Copy the input frame to the output frame
      frame_out = hdmi_out.newframe()
      frame_out[:,:,:] = orig_img[:,:,:]
      hdmi_out.writeframe(frame_out)
    else:
      break
end_time = time.time()

print('Runtime:',end_time-start_time,'FPS:', frame_cnt/(end_time-start_time))

hdmi_out.close()
videoIn.release()